## 准备数据

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.MNIST(
    root = './data/',
    train = True,                         
    transform = ToTensor()
)
test_data = datasets.MNIST(
    root = './data/', 
    train = False, 
    transform = ToTensor()
)

from torch.utils.data import DataLoader
loaders = {
    'train' : DataLoader(training_data,
                         batch_size=3000,
                         shuffle=True,
                         num_workers=1),
    
    'test'  : DataLoader(test_data, 
                         batch_size=100,
                         shuffle=True,
                         num_workers=1),
}

d:\miniconda\envs\py311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 建立模型

In [2]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()
        self.linear1 = torch.nn.Linear(28**2, 100)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(100, 10)
        self.logSoftmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        x = x.reshape([x.shape[0], -1])
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.logSoftmax(x)
        return x

# 求解器

In [3]:
class ModelSolver(object):
    def __init__(self, model, dataloaders, **kwargs):
        """
        Construct a new Solver instance.
        ## Required arguments:
        - model
        - dataloaders

        ## Optional arguments:

        ### training details
        - learning_rate(int): Learning rate of optimizer.
        - num_epochs(int): The number of epochs to run for during training.
        - weight_decay(float)

        ### output
        - print_every(int): Training losses will be printed every print_every
          iterations.
        - verbose(bool): Print training result or not.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = model
        self.model = self.model.to(self.device)
        self.dataloaders = dataloaders

        # Unpack keyword arguments
        # training stategies
        self.learning_rate = kwargs.pop("learning_rate", 1e-5)
        self.num_epochs = kwargs.pop("num_epochs", 10)
        weight_decay = kwargs.pop("weight_decay", 0.05)
        self.optim = torch.optim.Adam(self.model.parameters(), self.learning_rate, weight_decay=weight_decay)

        # output
        self.print_every = kwargs.pop("print_every", 1)
        self.verbose = kwargs.pop("verbose", True)

        # Throw an error if there are extra keyword arguments
        if len(kwargs) > 0:
            extra = ", ".join('"%s"' % k for k in list(kwargs.keys()))
            raise ValueError("Unrecognized arguments %s" % extra)

        self._reset()

    def _reset(self):
        """
        Set up some book-keeping variables for optimization. Don't call this
        manually.
        """
        # Set up some variables for book-keeping
        self.loss_history = []

    def _step(self, batch):
        """
        Make a single gradient update. This is called by train() and should not
        be called manually.
        """
        images = batch[0].to(self.device)
        labels = batch[1].to(self.device)
        output = self.model(images)
        loss = self.loss(output, labels)
        self.loss_history.append(loss.detach().cpu().numpy())
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()

    def train(self):
        """
        Run optimization to train the model.
        """
        self.model.train()

        iterations_per_epoch = max(len(self.dataloaders['train']), 1)
        num_iterations = self.num_epochs * iterations_per_epoch
        
        t = 0
        for e in range(self.num_epochs):
            # Update parameters
            for i, batch in enumerate(self.dataloaders['train']):
                self._step(batch)
                # Maybe print training loss
                if self.verbose and t % self.print_every == 0:
                    print(
                        "(Training Epoch %d Iteration %d / %d) loss: %f"
                        % (e + 1, t + 1, num_iterations, self.loss_history[-1])
                    )
                t += 1
            # Validation
            accuracy, loss = self.val()
            print("(Validation Epoch %d) loss: %f accuracy: %f" % (e, loss, accuracy))

    def val(self):
        """
        Run test.
        """
        accuracy_history = []
        loss_history = []
        self.model.eval()
        with torch.no_grad():
            for i, batch in enumerate(self.dataloaders['test']):

                images = batch[0].to(self.device)
                labels = batch[1].to(self.device)
                output = self.model(images)

                loss_history.append(self.loss(output, labels))
                accuracy_history.append(self.accuracy(output, labels))
                
        accuracy = sum(accuracy_history)/len(accuracy_history)
        loss = sum(loss_history)/len(loss_history)
        return accuracy, loss

    def loss(self, log_prob, labels):
        loss_function = torch.nn.NLLLoss()
        return loss_function(log_prob, labels)
    
    def accuracy(self, log_prob, labels):
        predictions = torch.argmax(log_prob, dim=-1)
        return torch.mean((predictions == labels).to(torch.float32))


## 实际训练

In [4]:
tinymodel = TinyModel()
solver = ModelSolver(tinymodel, 
                     loaders, 
                     num_epochs=5, 
                     learning_rate=1e-3,
                     print_every=5)
solver.train()

(Training Epoch 1 Iteration 1 / 100) loss: 2.305859
(Training Epoch 1 Iteration 6 / 100) loss: 2.085197
(Training Epoch 1 Iteration 11 / 100) loss: 1.824277
(Training Epoch 1 Iteration 16 / 100) loss: 1.554982
(Validation Epoch 0) loss: 1.315618 accuracy: 0.790100
(Training Epoch 2 Iteration 21 / 100) loss: 1.338567
(Training Epoch 2 Iteration 26 / 100) loss: 1.138084
(Training Epoch 2 Iteration 31 / 100) loss: 1.005445
(Training Epoch 2 Iteration 36 / 100) loss: 0.877851
(Validation Epoch 1) loss: 0.795927 accuracy: 0.848800
(Training Epoch 3 Iteration 41 / 100) loss: 0.810048
(Training Epoch 3 Iteration 46 / 100) loss: 0.770277
(Training Epoch 3 Iteration 51 / 100) loss: 0.766285
(Training Epoch 3 Iteration 56 / 100) loss: 0.718639
(Validation Epoch 2) loss: 0.675811 accuracy: 0.875800
(Training Epoch 4 Iteration 61 / 100) loss: 0.687512
(Training Epoch 4 Iteration 66 / 100) loss: 0.682928
(Training Epoch 4 Iteration 71 / 100) loss: 0.673706
(Training Epoch 4 Iteration 76 / 100) loss